### Importing all the required variables

In [0]:
%run "/Workspace/UPI Transactions/Databricks_Notebooks/ETL_Files/01_Config"

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold_upi.dim_banks (
  bank_id INTEGER,
  bank_name STRING
);

CREATE TABLE IF NOT EXISTS gold_upi.dim_category (
  category_id INTEGER,
  category STRING
);

CREATE TABLE IF NOT EXISTS gold_upi.dim_gateway (
  gateway_id INTEGER,
  payment_gateway STRING
);

CREATE TABLE IF NOT EXISTS gold_upi.dim_users (
  user_id STRING,
  user_name STRING
);

CREATE TABLE IF NOT EXISTS gold_upi.dim_proof (
  proof_id INTEGER,
  id STRING
);

CREATE TABLE IF NOT EXISTS gold_upi.dim_states (
  state_id INTEGER,
  state STRING
);

CREATE TABLE IF NOT EXISTS gold_upi.fact_transactions (
  user_id STRING,
  sender_bank_id INTEGER,
  sender_ifsc STRING,
  receiver_bank_id INTEGER,
  receiver_ifsc STRING,
  amount DOUBLE,
  gateway_id INTEGER,
  category_id INTEGER,
  state_id INTEGER,
  transaction_id STRING,
  status STRING,
  kyc BOOLEAN,
  proof_id INTEGER,
  timestamp TIMESTAMP,
  year INTEGER,
  month INTEGER
);

In [0]:
%sql
MERGE INTO gold_upi.dim_banks AS target
USING delta.`YOUR-SILVER-BANK-DIMENSION-FOLDER-LOCATION` AS source
ON target.bank_id = source.bank_id
WHEN NOT MATCHED THEN INSERT *

In [0]:
%sql
MERGE INTO gold_upi.dim_category AS target
USING delta.`YOUR-SILVER-CATEGORY-DIMENSION-FOLDER-LOCATION` AS source
ON target.category_id = source.category_id
WHEN NOT MATCHED THEN INSERT *;

In [0]:
%sql
MERGE INTO gold_upi.dim_gateway AS target
USING delta.`YOUR-SILVER-GATEWAY-DIMENSION-FOLDER-LOCATION` AS source
ON target.gateway_id = source.gateway_id
WHEN NOT MATCHED THEN INSERT *;

In [0]:
%sql
MERGE INTO gold_upi.dim_proof AS target
USING delta.`YOUR-SILVER-PROOF-DIMENSION-FOLDER-LOCATION` AS source
ON target.proof_id = source.proof_id
WHEN NOT MATCHED THEN INSERT *;

In [0]:
%sql
MERGE INTO gold_upi.dim_states AS target
USING delta.`YOUR-SILVER-STATES-DIMENSION-FOLDER-LOCATION` AS source
ON target.state_id = source.state_id
WHEN NOT MATCHED THEN INSERT *;

In [0]:
%sql
MERGE INTO gold_upi.dim_users AS target
USING delta.`YOUR-SILVER-USERS-DIMENSION-FOLDER-LOCATION` AS source
on target.user_id = source.user_id
WHEN NOT MATCHED THEN INSERT *;

In [0]:
df_fact = spark.read.format('delta').load(f"{s3_bucket}/{silver_folder}/fact_transactions/")

df_fact.write.format('delta').mode('overwrite')\
    .saveAsTable("workspace.gold_upi.fact_transactions")

In [0]:
%sql
SELECT ds.state, sum(ft.amount) AS total_transactions
FROM gold_upi.fact_transactions ft
LEFT JOIN gold_upi.dim_states ds
ON ft.state_id = ds.state_id
GROUP BY ds.state
ORDER BY total_transactions DESC

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT dc.category, count(ft.user_id) AS most_used_category,
sum(ft.amount) AS total_amount
FROM gold_upi.fact_transactions ft
LEFT JOIN gold_upi.dim_category dc
ON ft.category_id = dc.category_id
GROUP BY dc.category
ORDER BY most_used_category DESC, total_amount DESC

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT dg.payment_gateway, count(ft.transaction_id) AS count_of_people
FROM gold_upi.fact_transactions ft
LEFT JOIN gold_upi.dim_gateway dg
ON ft.gateway_id = dg.gateway_id
GROUP BY dg.payment_gateway
ORDER BY count_of_people DESC

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT ft.month, sum(ft.amount) as monthly_transactional_amoumt
FROM gold_upi.fact_transactions ft
GROUP BY ft.month
ORDER BY monthly_transactional_amoumt DESC

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT round((sum(
  case when ft.status = 'completed' then 1.0 else 0.0 END
) * 100) / count(*), 3) AS success_percent,
round((sum(
  case when ft.status = 'failed' then 1.0 else 0.0 END
) * 100) / count(*), 3) AS failed_percent
FROM gold_upi.fact_transactions ft